In [ ]:
import cobra
from cobra.io import save_json_model, read_sbml_model, write_sbml_model
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
mass_balance_dir = '/../4.mass_balance'
man_dir = '/../3.manual_check'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
# model_ori = read_sbml_model(f"{os.getcwd()}/{man_dir}/Salb-GEM-Man-gapfill.xml")
model = read_sbml_model(f"{os.getcwd()}/{mass_balance_dir}/Salb-GEM-Biosustain.xml")
# model_ori = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

# Get all gapfilled reactions

In [ ]:
diff_r = []
for r in model.reactions:
    if model_ori.reactions.has_id(r.id) == False:
        diff_r.append(r)
    else:
        if model_ori.reactions.get_by_id(r.id).bounds != r.bounds:
            diff_r.append(r)
        elif model_ori.reactions.get_by_id(r.id).build_reaction_string() != r.build_reaction_string():
            diff_r.append(r)

reaction_data = [{
    'Reaction ID': reaction.id,
    'Name': reaction.name,
    'Reaction String': reaction.reaction,
    'Reference': reaction.annotation
} for reaction in diff_r]

reaction_df = pd.DataFrame(reaction_data)
reaction_df.to_csv('all_gapfilled_reactions.csv')
print(reaction_df)

In [ ]:
# Find out what genes could be wrong by checking growth.

possibly_wrong = []
for r in diff_r:
    with model_ori:
        model_ori.add_reactions([r])
        model_ori.objective = 'growth'
        solution = model_ori.optimize().objective_value
        print(model_ori.optimize().fluxes[r.id])
    print(r, solution)
    if solution > 0.06:
        possibly_wrong.append(r)

possibly_wrong

In [ ]:
addition = list(set(diff_r) - set(possibly_wrong))

# Manual Check Gapfilled Reactions in all_gapfilled_reactions.csv

In [ ]:
# Update reaction names
for r in addition:
    if r.name == "":
        print(r.id)


In [ ]:
model.reactions.get_by_id('RE0691C').annotation

In [ ]:
model.metabolites.dnac_c.formula = 'R'
model.metabolites.dnac_c.charge = -1

model.metabolites.dna5mtc_c.formula = 'CH3R'
model.metabolites.asntrna_c.formula = 'C19H33N2O18P2'
model.metabolites.asntrna_c.charge = 2
model.metabolites.asptrna_asn_c.formula = 'C19H30NO19P2'

model.metabolites.get_by_id('2ahethmpp_c').formula = 'C14H23N4O8P2S'

model.metabolites.trnagln_c.formula = 'C15H20N5O10PR'
model.metabolites.glntrna_c.formula = 'C20H29N7O12PR'
model.metabolites.glntrna_c.charge = 1
model.metabolites.glutrna_gln_c.formula = 'C20H26N6O13PR'
model.metabolites.glutrna_gln_c.charge = -1


model.metabolites.sertrna_sec_c.formula = 'C18H28NO18P2'
model.metabolites.sertrna_sec_c.charge = -2
model.metabolites.sectrna_c.formula = 'C18H30NO17P2Se'

In [ ]:
for r in model.reactions:
    balance = r.check_mass_balance()
    if balance != {} and not (r.id.startswith('EX_') or r.id.startswith('DM_') or "PSEUDO" in r.id):
        print(r.id,r.name, balance)

In [ ]:
model.reactions.get_by_id('3HPCOAHYD').name = '3-hydroxyisobutyryl-CoA hydrolase'
model.reactions.CBMD.name = 'Carbamate deaminase'
model.reactions.SELGTHR.name = 'Selenate glutathione reductase'
model.reactions.get_by_id('RE0691C').name = 'putrescine—2-oxoglutarate transaminase'

In [ ]:
from cobra.io import write_sbml_model, save_json_model, save_matlab_model, save_yaml_model
model_dir = '../../../model/'
write_sbml_model(model, f'{model_dir}/Salb-GEM-Biosustain.xml')
save_json_model(model, f'{model_dir}/Salb-GEM-Biosustain.json')
save_matlab_model(model, f'{model_dir}/Salb-GEM-Biosustain.mat')
save_yaml_model(model, f'{model_dir}/Salb-GEM-Biosustain.yaml')

In [ ]:
model

In [ ]:
from cobra.flux_analysis.variability import flux_variability_analysis, find_essential_genes, find_blocked_reactions

# flux_variability_analysis(model, fraction_of_optimum=0.1)
find_blocked_reactions(model)
# find_essential_genes(model)

In [ ]:
from cobra.flux_analysis import fastcc

fastcc(model)